# HLT Assignment

In addition to practising the process of loading data, regularizing it and using it to train a model, the goal of this assignment is to investigat the effect of changing model parameters on the performance.

## Step 1: loading the data

We'll be going a bit further with the robot collision dataset. This time, instead of looking at just the first file, we'll look at all five different tasks combined (lp1.data to lp5.data). Prepare two different arrays, X1 and X2, as follows:
- Each element in X1 is the immediate reading of the force and torque values after an event, [f1, f2, f3, t1, t2, t3]. The first element should be [1, 1, 63, -3, -1, 0]
- Each element in X2 contains 18 values in total - the first, fifth and tenth sets of sensor readings after an event. The first element should be [-1, -1, 63, -3, -1, 0, -1, -1, 63, -3, -1, 0, -1, -1, 61, -3, 0, 0]

y should contain the corresponding classes, represented as integers according the the provided dictionary

In [1]:
files = ['robot_execution_failure/lp1.data', 'robot_execution_failure/lp2.data',
'robot_execution_failure/lp3.data', 'robot_execution_failure/lp4.data', 'robot_execution_failure/lp5.data']
classes = {'normal':0, 'collision':1, 'obstruction':2, 'fr_collision':3}

In [2]:
#load files data
f_data = [] #introduce empty array

#loop through the different files opening them and adding them into the empty array initiated
for i in range (len(files)):
    f_single = open(files[i])
    data = f_single.readlines()
    all_data = f_data.extend(data)
f_data[:2] #the first 2 lines

['normal\n', '\t-1\t-1\t63\t-3\t-1\t0\n']

In [3]:
X1 = [] # inputs
y = [] # true values
classes = {'normal':0, 'collision':1, 'obstruction':2, 'fr_collision':3}

# iterate over the lines of the file. If a line matches one of our classes, we split the next line 
# to get the six readings and use those as our features. 
for i in range(len(f_data) - 1):
    line = f_data[i].strip() # .strip() removes the line endings \n
    if line in classes.keys(): # If the line matches one of our classes (for eg, 'normal')
        features = [int(x) for x in f_data[i+1].strip().split('\t')] # Split the next line to get our features
        X1.append(features)
        y.append(classes[line]) # And record which class this set of features belongs to
X1[0] #show first element

[-1, -1, 63, -3, -1, 0]

In [4]:
X2 = [] # inputs
y = [] # true values
classes = {'normal':0, 'collision':1, 'obstruction':2, 'fr_collision':3}

# iterate over the lines of the file. If a line matches one of our classes, we split the next line 
# to get the six readings and use those as our features. 
for i in range(len(f_data) - 1):
    line = f_data[i].strip() # .strip() removes the line endings \n
    if line in classes.keys(): # If the line matches one of our classes (for eg, 'normal')
        #add the first, fifth and tenth sets of sensor readings after an event.
        features = [int(x) for x in f_data[i+1].strip().split('\t')] + [int(x) for x in f_data[i+5].strip().split('\t')] + [int(x) for x in f_data[i+10].strip().split('\t')]
        X2.append(features)
        y.append(classes[line]) # And record which class this set of features belongs to
X2[0] #show first element

[-1, -1, 63, -3, -1, 0, -1, -1, 63, -3, -1, 0, -1, -1, 61, -3, 0, 0]

## Step 2: establishing a baseline

Using techniques covered in this unit, split X1 and y into separate training and testing sets. Use the training set to train a neural network (MLPClassifier) using default parameters but with hidden_layer_sizes=(20, 20, 20). Use the test data you held back to score the model you have created. How well does it perform? Print out the score and confusion matrix. For more accuracy, run through these steps 10 times and find the average score - bonus points for running more times and getting a standard deviation!

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X1, y)

In [6]:
print(len(X_train))

201


In [7]:
# Preparing to scale the inputs
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [8]:
# Scaling the inputs 
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [9]:
from sklearn.neural_network import MLPClassifier
# Creating the neural network
mlp = MLPClassifier(hidden_layer_sizes=(20,20,20))

In [10]:
# Training
mlp.fit(X_train, y_train)

/Users/vivianopondoh/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(20, 20, 20), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [11]:
from sklearn.metrics import classification_report,confusion_matrix
predictions = mlp.predict(X_test)
print(confusion_matrix(y_test,predictions))

[[25  0  0  0]
 [ 9 14  0  0]
 [ 6  5  4  0]
 [ 5  0  0  0]]


In [12]:
print(mlp.score(X_test, y_test))

0.6323529411764706


In [19]:
# Score avrage
import pandas as pd 
df = pd.DataFrame({"X1_mean":[0.7058823529411765, 0.7205882352941176, 0.7352941176470589, 0.5441176470588235, 0.5588235294117647, 0.6764705882352942, 0.6323529411764706, 0.6911764705882353, 0.7647058823529411, 0.75]})
df.mean()

X1_mean    0.677941
dtype: float64

In [20]:
# Score std
df_20 = pd.DataFrame({"X1_mean":[0.7058823529411765, 0.7205882352941176, 0.7352941176470589, 0.5441176470588235, 0.5588235294117647, 0.6764705882352942, 0.6323529411764706, 0.6911764705882353, 0.7647058823529411, 0.75, 0.6323529411764706, 0.5735294117647058, 0.45588235294117646, 0.6470588235294118, 0.6470588235294118, 0.6176470588235294, 0.6764705882352942, 0.6323529411764706, 0.6764705882352942, 0.6323529411764706]})
df_20.std()

X1_mean    0.075273
dtype: float64

If you're getting a convergence warning, you could try having the model train over more iterations - change max_iter = 1000 or 10,000. Does this improve the average score?

## Step 3: adding more inputs

Use X2 in place of X1 - does the score increase or decrease? Was this what you expected? How many samples are there in our training set?

0.7058823529411765, 0.7205882352941176, 0.7352941176470589, 0.5441176470588235, 0.5588235294117647, 
0.6764705882352942, 0.6323529411764706, 0.6911764705882353, 0.7647058823529411, 0.75
0.6323529411764706, 0.5735294117647058, 0.45588235294117646, 0.6470588235294118, 0.6470588235294118
0.6176470588235294, 0.6764705882352942, 0.6323529411764706, 0.6764705882352942, 0.6323529411764706




## Step 4: feature engineering

Back to X1 as our input. Add an extra feature to each item in the array to represent the total force $f_t$. Assume:

$f_t^2 = f_1^2 + f_2^2 + f_3^2$

Your first input should now look like this:
X1[0] = [-1, -1, 61, -3, 0, 0, 61.0163912403872]

Repeat the steps from step 2. *Has this extra feature improved model performance?*

## Step 5: exploring

Create at least 3 more models, adding features or changing the number and size of the hidden layers. Print out the average score for your best model. Comment on what you've found.

In [ ]:
# Your answer